In [1]:
from rdkit import Chem
from rdkit.Chem import QED, RDConfig, Descriptors, DataStructs, AllChem

import pandas as pd
import os
import sys
sys.path.append(os.path.join(RDConfig.RDContribDir, 'SA_Score'))

import networkx as nx

/Users/jihyeonje/opt/anaconda3/envs/2020Woo/lib/python3.8/site-packages/scipy/__init__.py:143: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.19.0)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#dataset from https://github.com/wengong-jin/iclr19-graph2graph/tree/master
df_qed = pd.read_csv('/Users/jihyeonje/Desktop/data/qed/train_pairs.txt', header=None, sep=' ') #sim coef 0.4, qed from [0.7,0.8] to [0.9,1.0]
df_plogp = pd.read_csv('/Users/jihyeonje/Desktop/data/logp04/train_pairs.txt', header=None, sep=' ') #sim coef 0.4 

In [3]:
smi = df_qed[0][10]
mol = Chem.MolFromSmiles(smi)

#start processing 3D coordinates
updated_mol= Chem.AddHs(mol)
AllChem.EmbedMolecule(updated_mol)
AllChem.UFFOptimizeMolecule(updated_mol)
updated_mol.GetConformer()

#to get atom type, coordinate, edges
mblock = Chem.MolToMolBlock(updated_mol)
print(mblock)


     RDKit          3D

 47 49  0  0  0  0  0  0  0  0999 V2000
   -5.6686   -1.5782    0.5171 C   0  0  0  0  0  0  0  0  0  0  0  0
   -4.2729   -1.2484   -0.0317 C   0  0  0  0  0  0  0  0  0  0  0  0
   -4.2224   -1.4862   -1.5518 C   0  0  0  0  0  0  0  0  0  0  0  0
   -3.8208    0.1776    0.3937 C   0  0  2  0  0  0  0  0  0  0  0  0
   -4.7142    1.2276   -0.1176 N   0  0  0  0  0  0  0  0  0  0  0  0
   -4.6984    2.5614    0.4101 C   0  0  0  0  0  0  0  0  0  0  0  0
   -5.5853    3.5478   -0.1154 N   0  0  0  0  0  0  0  0  0  0  0  0
   -3.9038    2.8716    1.3383 O   0  0  0  0  0  0  0  0  0  0  0  0
   -2.3957    0.4387   -0.0293 C   0  0  0  0  0  0  0  0  0  0  0  0
   -2.1631    1.0991   -1.0790 O   0  0  0  0  0  0  0  0  0  0  0  0
   -1.3123   -0.0778    0.7596 N   0  0  0  0  0  0  0  0  0  0  0  0
    0.0514   -0.1384    0.3102 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.3706   -0.4037   -1.0495 C   0  0  0  0  0  0  0  0  0  0  0  0
    1.7116   -0.4847   -1

In [4]:
parsed = mblock.split("\n")
stats = parsed[3]
tot_atoms = int(stats.split(" ")[1])
tot_edges = int(stats.split(" ")[2])

coors = parsed[4:tot_atoms+4]
edges = parsed[tot_atoms+4:tot_atoms+tot_edges+4]


In [5]:
edge_dict = []
for i in range(tot_edges):
    edge_inf = [int(x) for x in edges[i].split()[0:3]]
    edge_dict.append({(edge_inf[0], edge_inf[1]):edge_inf[2]})

In [6]:
node_dict = []
for j in range(tot_atoms):
    atom_type = coors[j][:-37][-1]
    xyz_coor = [float(x) for x in coors[j][:-37][:-2].split()]
    node_dict.append({'node': j+1, 'x': xyz_coor[0], 'y': xyz_coor[1], 'z': xyz_coor[2], 'type': atom_type})

In [7]:
N = len(node_dict)
N

47

In [8]:
import igraph as ig

In [9]:
edges = [[*edge_dict[k]][0] for k in range(len(edge_dict))]
G = ig.Graph(edges, directed=False)

In [10]:
node_num=[]
atom_type=[]
Xn = []
Yn = []
Zn = []
for node in node_dict:
    node_num.append(node['node'])
    atom_type.append(node['type'])
    Xn.append('x')
    Yn.append('y')
    Zn.append('z')

In [11]:
layt=G.layout('kk', dim=3)


In [12]:

Xe=[]
Ye=[]
Ze=[]
for e in edges:
    Xe+=[layt[e[0]][0],layt[e[1]][0], None]# x-coordinates of edge ends
    Ye+=[layt[e[0]][1],layt[e[1]][1], None]
    Ze+=[layt[e[0]][2],layt[e[1]][2], None]

In [13]:
import plotly.graph_objs as go

trace1=go.Scatter3d(x=Xe,
               y=Ye,
               z=Ze,
               mode='lines',
               line=dict(color='rgb(125,125,125)', width=1)
               )

trace2=go.Scatter3d(x=Xn,
               y=Yn,
               z=Zn,
               mode='markers',
               name='actors',
               marker=dict(symbol='circle',
                             size=6,
                             line=dict(color='rgb(50,50,50)', width=0.5)
                             )
               )

axis=dict(showbackground=False,
          showline=False,
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title=''
          )

layout = go.Layout(
         title="Network of coappearances of characters in Victor Hugo's novel<br> Les Miserables (3D visualization)",
         width=1000,
         height=1000,
         showlegend=False,
         scene=dict(
             xaxis=dict(axis),
             yaxis=dict(axis),
             zaxis=dict(axis),
        ),
     margin=dict(
        t=100
    ),
    hovermode='closest',
     )

In [21]:
data=[trace1, trace2]
fig=go.Figure(data=data, layout=layout)
import chart_studio.plotly as py

py.plot(fig, filename='Les-Miserables')